In [5]:
%matplotlib inline
import obspy
from obspy import read
import glob
import os, sys
import librosa
import numpy as np
from scipy import stats
import scipy as sp
import scipy.signal as signal
import pandas as pd
import matplotlib.pyplot as plt
import librosa.display
from sklearn.preprocessing import StandardScaler

random_state = 6
np.random.seed(random_state)

In [2]:
def extract_features(seismogram, signal_label):
    data = stats.zscore(seismogram.data)
    sample_rate = seismogram.stats.sampling_rate
    stft = np.abs(librosa.stft(data))
    mfccs = librosa.feature.mfcc(y = data, sr=sample_rate, n_mfcc=40)
    mfccs_mean = np.mean(mfccs.T, axis = 0)
    mfccs_delta = np.mean(librosa.feature.delta(mfccs).T, axis=0)
    
    # 12 chroma
    chroma = librosa.feature.chroma_stft(S = stft, sr = sample_rate)
    chroma_mean = np.mean(chroma.T, axis = 0)
    
    ## mel from melspectrogram
    # this feature does not add anything to improve model. Therefore I delete this features from the 
    # feature list
    
    # mel = np.mean(librosa.feature.melspectrogram(data, sr = sample_rate).T, axis= 0)
    
    ## spectral centroid
    S, phase = librosa.magphase(stft)
    centiroid =np.mean(librosa.feature.spectral_centroid(S=S)) # 1 feature
    max_amplitude = np.amax(S)
    mean_amplitude = np.mean(S)
    dbamplitude= np.mean(librosa.amplitude_to_db(S, ref=np.max).T, axis = 0) # 1025 features
    
    ## statistical parameters
    moment = sp.stats.moment(data)
    variation = sp.stats.variation(data)
    skew = sp.stats.skew(data)
    var = np.var(data)
    autocr = np.correlate(data, data)
    kurto = sp.stats.kurtosis(data)
    
    features = np.hstack([mfccs_mean, chroma_mean, centiroid, max_amplitude, mean_amplitude, moment, variation, skew, var, autocr, kurto, signal_label])
    
    return features

def parse_and_stack_seismograms(seismograms, label):
    
    features = np.empty((0,62))
    
    if label == 'earthquake':
        target = 0
    elif label == 'explosion':
        target = 1

    for i, seismogram in enumerate(seismograms):
        single_feature= extract_features(seismogram, target)
        features = np.vstack([features, single_feature])
        
    dataFrame = pd.DataFrame(features)
    return dataFrame

In [3]:
parent_dir_explosions = 'seismogram_v2/explosions/'
sub_dirs_explosions = ['1998-05-11-mb52-india','1998-05-28-mb48-pakistan', '1998-05-30-mb46-pakistan', '2013-02-12-mb51-north-korea', '2016-01-06-mb51-north-korea', '2017-09-03-mb63-north-korea']
parent_dir_earthquake = 'seismogram_v2/earthquakes/'
sub_dirs_earthquake = ['2004-12-26-mw90-sumatra', '2010-03-12-mw55-myanmar-india-border-region', '2017-08-15-mb49-southeast-of-ryukyu-islands', '2017-09-08-mww81-near-coast-of-chiapas-mexico', '2017-10-24-mww67-banda-sea']

## Explosions
print('--- Earthquake-----: ')
total = 0.0
for sub_dir in sub_dirs_earthquake:
    counter = len(glob.glob(os.path.join(parent_dir_earthquake, sub_dir, '*.SAC')))
    total += counter
    print('{}: {}'.format(sub_dir, counter))
print('Total seismograms: {}'. format(total))


## Explosions
print('--- Explosions-----: ')
total = 0.0
for sub_dir in sub_dirs_explosions:
    counter = len(glob.glob(os.path.join(parent_dir_explosions, sub_dir, '*.SAC')))
    total += counter
    print('{}: {}'.format(sub_dir, counter))
print('Total seismograms: {}'. format(total))

--- Earthquake-----: 
2004-12-26-mw90-sumatra: 1193
2010-03-12-mw55-myanmar-india-border-region: 3021
2017-08-15-mb49-southeast-of-ryukyu-islands: 2644
2017-09-08-mww81-near-coast-of-chiapas-mexico: 2722
2017-10-24-mww67-banda-sea: 3382
Total seismograms: 12962.0
--- Explosions-----: 
1998-05-11-mb52-india: 459
1998-05-28-mb48-pakistan: 470
1998-05-30-mb46-pakistan: 398
2013-02-12-mb51-north-korea: 3763
2016-01-06-mb51-north-korea: 2640
2017-09-03-mb63-north-korea: 2804
Total seismograms: 10534.0


In [4]:
earthquakes = read('./seismogram_v2/earthquakes/**/*.SAC')
explosions = read('./seismogram_v2/explosions/**/*.SAC')

df_explosions  = parse_and_stack_seismograms(explosions, 'explosion')
df_earthquakes  = parse_and_stack_seismograms(earthquakes, 'earthquake')

/usr/local/lib/python3.6/site-packages/obspy/core/trace.py:187: UserWarning: Calibration factor set to 0.0!
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.6/site-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
/usr/local/lib/python3.6/site-packages/scipy/stats/stats.py:977: RuntimeWarning: divide by zero encountered in float_scalars
  return a.std(axis) / a.mean(axis)
/usr/local/lib/python3.6/site-packages/scipy/stats/stats.py:2247: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


ParameterError: Audio buffer is not finite everywhere

In [7]:
def get_column_names():
    features = {
        'mfccs': 40,
        'chroma': 12,
        'centiroid': 1,
        'max_amplitude':1, 
        'mean_amplitude':1,
        'moment': 1,
        'variation': 1, 
        'skew': 1, 
        'var': 1, 
        'autocr': 1, 
        'kurto': 1, 
        'target': 1
    }
    
    names = list(features.keys())
    val = list(features.values())
    
    columns = []
    
    for i in range(len(features)):
        if val[i] > 1:
            for j in range(val[i]):
                columns.append(str(names[i])+'_'+ str(j))
                
        else:
            columns.append(str(names[i]))
    
    return columns

In [8]:
frames = [df_explosions, df_earthquakes]
df = pd.concat(frames)
df.columns = get_column_names()
df.describe()

,mfccs_0,mfccs_1,mfccs_2,mfccs_3,mfccs_4,mfccs_5,mfccs_6,mfccs_7,mfccs_8,mfccs_9,...,centiroid,max_amplitude,mean_amplitude,moment,variation,skew,var,autocr,kurto,target
count,22216.000000,22216.000000,22216.000000,22216.000000,22216.000000,22216.000000,22216.000000,22216.000000,22216.000000,22216.000000,...,22216.000000,2.221600e+04,2.221600e+04,22216.0,22216.000000,22216.000000,2.221600e+04,2.221600e+04,22216.000000,22216.000000
mean,841.535354,72.717304,-1.120552,52.234151,2.483228,34.444831,7.671640,22.622343,12.085533,15.528755,...,971.449960,2.398654e+08,2.960831e+05,0.0,20.183022,-0.027121,5.556371e+11,9.640448e+16,6.574073,0.450801
std,191.235047,38.199436,39.099088,19.406940,24.244137,12.987450,15.622192,9.220608,10.365557,7.056197,...,1035.548479,1.618594e+09,4.228556e+06,0.0,4024.576118,1.393490,2.625937e+13,3.678673e+18,197.093498,0.497585
min,480.726115,-87.232311,-198.422667,-32.555981,-88.160009,-41.837174,-48.987699,-26.721758,-35.454891,-23.973469,...,3.588867,4.490561e+03,4.140245e+01,0.0,-427348.312500,-148.525772,0.000000e+00,1.098498e+06,-3.000000,0.000000
25%,699.059195,44.646795,-28.657048,39.179819,-16.100590,25.823460,-3.387524,16.973561,5.998598,11.365791,...,198.914251,5.850580e+05,1.836427e+03,0.0,-0.326106,-0.084359,3.699506e+04,1.065173e+10,-0.067151,0.000000
50%,823.266826,73.334659,2.453667,53.082876,3.731802,34.055244,9.972462,22.042250,14.128136,15.497889,...,550.956333,1.193249e+06,3.733953e+03,0.0,0.204730,-0.002923,1.485523e+05,4.448388e+10,0.197497,0.000000
75%,959.873963,99.011031,28.346150,64.883821,22.434741,42.256966,20.350147,27.610037,19.563121,19.315831,...,1465.813517,7.911418e+06,2.024887e+04,0.0,0.901340,0.064298,3.477907e+06,1.656055e+12,2.471488,1.000000
max,2065.265843,233.043814,112.815778,132.919902,61.106265,118.624193,58.102843,71.468831,53.939397,65.529958,...,6894.203309,7.142860e+10,5.437551e+08,0.0,287691.593750,67.194862,3.695033e+15,4.833089e+20,27424.243210,1.000000


In [9]:
df.to_csv('seismogram_data_62_new.csv', index=False)